In [89]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [90]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/dennissmith0/Running_Performance/main/prep/training_data.csv')

In [91]:
# # Display the first few rows of the dataframe
# print(df.head())

# # Get a summary of the dataframe
# print(df.info())

# # Generate descriptive statistics of the dataframe
# print(df.describe())

# # Check for missing values
# print(df.isnull().sum())

In [92]:
# Convert the "Activity Date" column to datetime
df['Activity Date'] = pd.to_datetime(df['Activity Date'])

# Sort data by date just to be sure
df = df.sort_values('Activity Date')

# Calculate days between activities and create new column that serves as measure for 'Rest Days'.
# NOTE: this measure reflects ALL activities prior to the next function which removes all activities that are not a 'Run'
df['Days Between Activity'] = df['Activity Date'].diff().dt.days

# First value = NaN, replace with 0.
if pd.isna(df.loc[0, 'Days Between Activity']):
    df.loc[0, 'Days Between Activity'] = 0.0


# Now set as datetime index
df.set_index('Activity Date', inplace=True)

In [93]:
# For now, remove all activities that are not a Run
df = df[df['Activity Type'].isin(['Run'])] #.reset_index(drop=True)
# Drop the column
df.drop(columns=['Activity Type'], inplace=True)

<ipython-input-93-591deb98c096>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Activity Type'], inplace=True)


In [94]:
# Replace NaN values in the 'Average Speed' column with the calculated speed using the 'Distance.1' and 'Moving Time' columns
def replace_nan_average_speed(df):
    df['Average Speed'] = df.apply(lambda row: row['Distance.1'] / row['Moving Time'] if pd.isna(row['Average Speed']) else row['Average Speed'], axis=1)
    return df

# Replace NaN values in 'Average Speed' column
df = replace_nan_average_speed(df)

<ipython-input-94-8de81e79b4a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Average Speed'] = df.apply(lambda row: row['Distance.1'] / row['Moving Time'] if pd.isna(row['Average Speed']) else row['Average Speed'], axis=1)


In [95]:
df.drop(columns=['Elapsed Time', 'Activity ID', 'Activity Name', 'Media', 'Commute', 'From Upload', 'Filename', 'Athlete Weight',
                 'Activity Gear', 'Number of Runs', 'Prefer Perceived Exertion',
                 'Average Temperature', 'Elevation Loss', 'Gear'], inplace=True)

# Drop potentially important features with high NaN columns for now, that we intend to find a measure that fills the value in a meaningful way.
df.drop(columns=['Grade Adjusted Distance'], inplace=True)

<ipython-input-95-b3944c4c312f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Elapsed Time', 'Activity ID', 'Activity Name', 'Media', 'Commute', 'From Upload', 'Filename', 'Athlete Weight',
<ipython-input-95-b3944c4c312f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Grade Adjusted Distance'], inplace=True)


In [96]:
def remove_columns_with_prefix(df, prefix):
    columns_to_remove = [column for column in df.columns if column.startswith(prefix)]
    df.drop(columns=columns_to_remove, inplace=True)
    return df

df = remove_columns_with_prefix(df, '<span')


<ipython-input-96-2471a2ac00d1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_to_remove, inplace=True)


In [97]:
# Some columns are repeats with more decimals, keep the rounded up decimals columns, remove the extras
def drop_columns_with_extra_decimals(df, suffix):
    columns_with_extra_decimals = [column for column in df.columns if column.endswith(suffix)]
    df.drop(columns=columns_with_extra_decimals, inplace=True)
    return df

df = drop_columns_with_extra_decimals(df, '.1')

<ipython-input-97-982a84b70c81>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_with_extra_decimals, inplace=True)


In [98]:
# Run for dropping each column with empty columns or 50% NaN values
def drop_columns_with_null(df):
    columns_with_null = df.columns[df.isnull().sum() >= (.50 * len(df))]
    df.drop(columns=columns_with_null, inplace=True)
    return df

df = drop_columns_with_null(df)

In [99]:
def remove_columns_with_few_values(dataframe, threshold):
    columns_to_remove = [column for column in dataframe.columns if dataframe[column].count() < threshold]
    dataframe.drop(columns=columns_to_remove, inplace=True)
    return dataframe

# Remove columns with less than 1000 values
# df = remove_columns_with_few_values(df, 1000)
df = remove_columns_with_few_values(df, 50)

In [100]:
df.isnull().sum()

Distance                   0
Max Heart Rate           163
Relative Effort          163
Moving Time                0
Max Speed                  5
Average Speed              0
Elevation Gain             2
Elevation Low             23
Elevation High            23
Max Grade                  5
Average Grade              0
Max Cadence               44
Average Cadence           20
Average Heart Rate       305
Calories                   0
Days Between Activity      0
dtype: int64

In [101]:
df[df['Average Heart Rate'].isna()].tail(20)

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity
Activity Date,,,,,,,,,,,,,,,,
2020-09-17 21:20:51,5.62,NaN,NaN,3060.0,4.900000,1.838301,478.000000,254.000000,540.000000,49.099998,-0.039110,124.0,68.241158,NaN,392.000000,0.0
2020-09-19 15:21:32,40.67,NaN,NaN,23386.0,5.300000,1.739246,1681.000000,206.399994,944.400024,42.700001,0.994739,124.0,71.098877,NaN,2730.000000,1.0
2020-09-23 20:46:49,7.35,NaN,NaN,2406.0,5.800000,3.055736,60.000000,252.800003,313.399994,8.300000,-0.038084,99.0,84.018570,NaN,578.000000,0.0
2020-09-26 15:14:37,27.42,NaN,NaN,16500.0,4.800000,1.662358,1533.000000,656.599976,1156.199951,43.599998,0.382079,123.0,66.662773,NaN,1826.000000,0.0
2020-10-10 14:14:10,3.55,NaN,NaN,2500.0,4.800000,1.419840,152.000000,266.799988,346.000000,29.700001,0.078882,91.0,74.296761,NaN,251.000000,1.0
2020-10-10 16:54:33,8.25,NaN,NaN,3889.0,4.700000,2.122139,397.000000,269.399994,557.000000,37.200001,0.189022,116.0,74.842094,NaN,596.000000,0.0
2020-10-12 16:21:36,22.38,NaN,NaN,12689.0,5.400000,1.763732,1227.000000,242.199997,579.599976,48.099998,0.138517,120.0,72.773270,NaN,1599.000000,1.0
2020-10-15 15:23:39,14.85,NaN,NaN,7048.0,6.600000,2.107080,675.000000,468.000000,946.200012,48.000000,0.106392,115.0,73.636322,NaN,1041.000000,0.0
2020-10-21 13:42:34,20.92,NaN,NaN,11518.0,7.700000,1.816314,1196.000000,469.200012,1234.400024,44.200001,1.228472,122.0,72.806862,NaN,1415.000000,1.0


In [102]:
df.tail(10)

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity
Activity Date,,,,,,,,,,,,,,,,
2023-06-08 14:52:08,9.40,171.0,64.0,3531.0,5.267969,2.663234,289.0,1324.199951,1454.199951,47.197281,-0.085071,90.0,81.420311,147.338272,750.0,0.0
2023-06-09 13:00:10,17.73,167.0,73.0,11388.0,5.634082,1.556976,1148.0,1707.199951,2534.399902,48.636837,-0.171453,118.0,67.808128,123.561058,1510.0,0.0
2023-06-11 00:51:20,3.63,168.0,21.0,1257.0,5.429980,2.893882,122.0,1571.000000,1676.000000,33.101528,-0.291399,88.0,79.991745,147.546112,270.0,1.0
2023-06-11 12:32:02,9.78,174.0,74.0,5362.0,6.736133,1.825179,720.0,1724.199951,2549.600098,49.164585,-0.128748,112.0,75.091339,142.909683,1036.0,0.0
2023-06-12 12:22:16,4.49,162.0,18.0,1917.0,5.833984,2.345144,89.0,1319.400024,1347.400024,29.658110,-0.320311,118.0,78.520119,140.439209,380.0,0.0
2023-06-16 01:02:30,5.64,173.0,37.0,2030.0,7.637750,2.781005,53.0,1294.599976,1343.199951,35.065315,-0.170048,110.0,82.424431,148.805817,430.0,1.0
2023-06-17 17:26:54,20.06,177.0,154.0,6998.0,5.051595,2.867208,134.0,1772.199951,2570.000000,30.338728,-3.919317,119.0,82.241585,153.711639,1513.0,1.0
2023-06-20 00:19:39,9.62,170.0,34.0,7086.0,5.224023,1.357962,542.0,1759.000000,2336.199951,49.868637,0.301376,122.0,62.268711,118.441292,893.0,2.0
2023-06-20 14:54:25,10.41,155.0,27.0,7866.0,4.337891,1.324174,743.0,1908.199951,2707.600098,44.702003,0.011521,125.0,58.039753,112.808022,1045.0,0.0


In [103]:
# SAVE FOR LATER MODEL. This model will only be modeled on "Run"

# # To adjust the "Activity Type" column to eliminate activities with less than 10% of the max value and convert it into
# # separate indicator columns for each activity, you can use the following function:
# def adjust_and_convert_activity_type_column(df):
#     activity_counts = df['Activity Type'].value_counts()
#     max_value = activity_counts.max()
#     threshold = max_value * 0.1

#     filtered_activities = activity_counts[activity_counts >= threshold].index.tolist()

#     # If the activity does not make the threshold, remove the activity (we are considering these activities as "outliers", with little effect on training)
#     df.loc[~df['Activity Type'].isin(filtered_activities), 'Activity Type'] = None
#     df.dropna(subset=['Activity Type'], inplace=True)


#     for activity in filtered_activities:
#         df[activity] = df['Activity Type'].apply(lambda x: 1 if x == activity else 0)

#     df.drop(columns=['Activity Type'], inplace=True)

#     return df

# # Adjust and convert the "Activity Type" column. This will allow the model to capture the influence of different activities on running performance.
# df = adjust_and_convert_activity_type_column(df)

In [104]:
# To impute missing values in the "Average Heart Rate" column with a random value within a range around the average, you can use the following function:
def impute_average_heart_rate(df):
    average_hr = df['Average Heart Rate'].mean()

    # Generate random values within the range of average_hr ± 10
    random_values = np.random.uniform(average_hr - 10, average_hr + 10, size=df['Average Heart Rate'].isnull().sum())

    # Replace missing values with the generated random values
    df.loc[df['Average Heart Rate'].isnull(), 'Average Heart Rate'] = random_values

    return df

# Impute missing values in "Average Heart Rate" column
df = impute_average_heart_rate(df)

In [105]:
# Feature Engineering: Training Stress
# For simplicity, let's define training stress as distance * average heart rate

# Convert 'Distance' and 'Average Heart Rate' columns to numeric types
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
df['Average Heart Rate'] = pd.to_numeric(df['Average Heart Rate'], errors='coerce')

Training_Stress_Space = df['Distance'] * df['Average Heart Rate']
Training_Stress_Time = (df['Moving Time'] / 60) * df['Average Heart Rate']
#df['Training Stress'] = Training_Stress_Time / Training_Stress_Space
# Check if distance is zero, assign Training Stress Time value if true, else calculate Training Stress
df['Training Stress'] = np.where(df['Distance'] == 0, Training_Stress_Time, Training_Stress_Time / Training_Stress_Space) # but now its a ridiciluously high score...

# But note rows that are workouts, e.g, not a run, the stress is much higher. Is this true?

In [106]:
# FEATURE:
# if add measure of days between activites, do so before removing activities that are below threshold percentage of max activity

# for imputing average heart rate value: TO DO: look at the activity type first, then get the average of those types.

In [107]:
df['Max Heart Rate'].isnull().sum()

163

In [108]:
df['Relative Effort'].isnull().sum()

163

In [109]:
df.columns

Index(['Distance', 'Max Heart Rate', 'Relative Effort', 'Moving Time',
       'Max Speed', 'Average Speed', 'Elevation Gain', 'Elevation Low',
       'Elevation High', 'Max Grade', 'Average Grade', 'Max Cadence',
       'Average Cadence', 'Average Heart Rate', 'Calories',
       'Days Between Activity', 'Training Stress'],
      dtype='object')

In [110]:
# # Removing for now, given that it leads to 'ValueError: Input X contains infinity or a value too large for dtype('float64').' when making predictions
# # Do we need? Given that 'Average Speed' = meters/second

# def calculate_average_pace(df):
#     # Convert moving time to minutes
#     moving_time_minutes = df['Moving Time'] / 60

#     # Convert distance from kilometers to miles
#     distance_miles = df['Distance'] * 0.621371

#     # # Reset the index of the dataframe
#     # df.reset_index(drop=True, inplace=True)
#     # # # Calculate average pace in minutes per mile
#     # # if df['Run'] == 1:
#     # #   df['Average Pace (min/mile)'] = moving_time_minutes / distance_miles
#     #     # Calculate average pace in minutes per mile only for 'run' activities
#     # df.loc[df['Run'] == 1, 'Average Pace (min/mile)'] = moving_time_minutes / distance_miles
#     # Calculate average pace in minutes per mile
#     df['Average Pace (min/mile)'] = moving_time_minutes / distance_miles


#     return df

# # Calculate the average pace
# df = calculate_average_pace(df)

In [111]:
df.tail(10)

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress
Activity Date,,,,,,,,,,,,,,,,,
2023-06-08 14:52:08,9.40,171.0,64.0,3531.0,5.267969,2.663234,289.0,1324.199951,1454.199951,47.197281,-0.085071,90.0,81.420311,147.338272,750.0,0.0,6.260638
2023-06-09 13:00:10,17.73,167.0,73.0,11388.0,5.634082,1.556976,1148.0,1707.199951,2534.399902,48.636837,-0.171453,118.0,67.808128,123.561058,1510.0,0.0,10.705020
2023-06-11 00:51:20,3.63,168.0,21.0,1257.0,5.429980,2.893882,122.0,1571.000000,1676.000000,33.101528,-0.291399,88.0,79.991745,147.546112,270.0,1.0,5.771350
2023-06-11 12:32:02,9.78,174.0,74.0,5362.0,6.736133,1.825179,720.0,1724.199951,2549.600098,49.164585,-0.128748,112.0,75.091339,142.909683,1036.0,0.0,9.137696
2023-06-12 12:22:16,4.49,162.0,18.0,1917.0,5.833984,2.345144,89.0,1319.400024,1347.400024,29.658110,-0.320311,118.0,78.520119,140.439209,380.0,0.0,7.115813
2023-06-16 01:02:30,5.64,173.0,37.0,2030.0,7.637750,2.781005,53.0,1294.599976,1343.199951,35.065315,-0.170048,110.0,82.424431,148.805817,430.0,1.0,5.998818
2023-06-17 17:26:54,20.06,177.0,154.0,6998.0,5.051595,2.867208,134.0,1772.199951,2570.000000,30.338728,-3.919317,119.0,82.241585,153.711639,1513.0,1.0,5.814224
2023-06-20 00:19:39,9.62,170.0,34.0,7086.0,5.224023,1.357962,542.0,1759.000000,2336.199951,49.868637,0.301376,122.0,62.268711,118.441292,893.0,2.0,12.276507
2023-06-20 14:54:25,10.41,155.0,27.0,7866.0,4.337891,1.324174,743.0,1908.199951,2707.600098,44.702003,0.011521,125.0,58.039753,112.808022,1045.0,0.0,12.593660


For now, in lieu of adding conditions that adjust the training stress score for activities such as yoga and rock climbing, lets remove these activities/rows with really high values.

---



In [112]:
# # Don't need in Version 1.0 of model as we only have 'Run' activity type
# df['Training Stress'].max()
# # This code filters the dataframe to keep only the rows where the 'Training Stress' value is less than or equal to 500. The resulting filtered dataframe is assigned back to the variable df, effectively removing the rows with high 'Training Stress' scores.
# df = df[df['Training Stress'] <= 500]


#TO-DO:
Make this function generate random values within a window of the average, rather than a constant average value for all NaN.

For fine-tuning: look at other features, and find similar values in other rows, and then impute comparable values for comparable runs.

In [113]:
# You can use the following function to check the entire dataframe for NaN values and impute the average value of each column:
def impute_nan_with_average(df):
    for column in df.columns:
        if df[column].isnull().any():
            average = df[column].mean()
            df[column].fillna(average, inplace=True)
    return df

# Impute NaN values with the average value of each column
df = impute_nan_with_average(df)

In [114]:
df.isnull().sum()

Distance                 0
Max Heart Rate           0
Relative Effort          0
Moving Time              0
Max Speed                0
Average Speed            0
Elevation Gain           0
Elevation Low            0
Elevation High           0
Max Grade                0
Average Grade            0
Max Cadence              0
Average Cadence          0
Average Heart Rate       0
Calories                 0
Days Between Activity    0
Training Stress          0
dtype: int64

# New Section

---



In [115]:
# FEATURE ENGINEERING: Training Session Intensity

# Firstly, it's advisable to normalize these features so that they are on a similar scale. This can be done using Min-Max normalization which scales the features to be between 0 and 1.
features_to_normalize = ['Distance', 'Relative Effort', 'Elevation Gain', 'Moving Time']

for feature in features_to_normalize:
    df[feature + "_norm"] = (df[feature] - df[feature].min()) / (df[feature].max() - df[feature].min())

# Create the 'Training Session Intensity' as a weighted sum of these features
# Here we are giving equal weight (0.25) to each of the four features, but you could adjust these weights based on what you think contributes more to the intensity of a training session.
# For example, if you think 'Relative Effort' is a more important factor, you might give it a higher weight.
df['Training Session Intensity'] = (df['Distance_norm'] * 0.25
                                    + df['Relative Effort_norm'] * 0.25
                                    + df['Elevation Gain_norm'] * 0.25
                                    + df['Moving Time_norm'] * 0.25)

# Drop the normalized columns, as they were just intermediates for this calculation:
df.drop([feature + "_norm" for feature in features_to_normalize], axis=1, inplace=True)

***Cumulative Load: ***
This is a measure of the total stress placed on your body over time, which we can calculate as a rolling sum of training session intensity over a certain window of time.

In [116]:
# FEATURE ENGINEERING: Cumulative Load
  # Dependent on Training Session Intensity

# "Cumulative Load" is a rolling sum of the training session intensity over a certain period of time.
# To calculate this, you can use the pandas rolling function, which provides rolling window calculations.

# Set the window size for cumulative load calculation
window_size = 7 # for example, to represent a week

# Calculate Cumulative Load
df['Cumulative Load'] = df['Training Session Intensity'].rolling(window=window_size).sum()

# Replace the NaN values for the first 6 rows
for i in range(6):
    df.loc[df.index[i], 'Cumulative Load'] = df.loc[df.index[:i+1], 'Training Session Intensity'].sum()

In [117]:
df[df['Training Stress'] == df['Training Stress'].max()]

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress,Training Session Intensity,Cumulative Load
Activity Date,,,,,,,,,,,,,,,,,,,
2021-06-25 18:50:53,10.98,174.0,87.0,28028.0,5.6,0.391858,1019.799988,1619.599976,2135.199951,50.0,-0.014568,123.0,55.943935,141.681915,3812.0,0.0,42.544019,0.316538,1.69603


In [118]:
df[df['Training Session Intensity'] == df['Training Session Intensity'].max()]

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress,Training Session Intensity,Cumulative Load
Activity Date,,,,,,,,,,,,,,,,,,,
2021-06-20 10:13:49,54.92,170.0,318.0,48885.0,6.4,1.123455,2905.0,1502.199951,2992.800049,49.599998,1.125999,124.0,62.593277,133.377197,5924.0,0.0,14.835215,0.909719,1.635248


In [119]:
df[df['Cumulative Load'] == df['Cumulative Load'].max()]

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress,Training Session Intensity,Cumulative Load
Activity Date,,,,,,,,,,,,,,,,,,,
2022-10-07 15:30:33,20.16,148.0,47.0,14769.0,4.491797,1.365061,936.0,2476.800049,3335.199951,46.685513,0.096227,124.0,58.351486,117.221138,1700.0,1.0,12.209821,0.263563,2.164077


In [120]:
df[df['Distance'] == df['Distance'].max()]

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress,Training Session Intensity,Cumulative Load
Activity Date,,,,,,,,,,,,,,,,,,,
2022-06-20 10:20:19,56.71,177.0,196.0,40257.0,5.733594,1.408883,3052.0,1497.800049,3022.600098,49.992821,1.075508,120.0,64.602371,133.462967,5225.0,0.0,11.831247,0.811154,1.76888


In [121]:
# FEATURE ENGINEERING: Performance/Readiness Score

# Engineer a performance metric that is a function of:
# Recent training load (e.g., the 'Cumulative Load' we just created)
# Rest days ('Days Between Activity')
# Intensity of the workouts ('Training Session Intensity')
# Other physiological metrics available in your data (like heart rate, speed, etc.)

# Define weights for each factor
weight_cumulative_load = 0.4
weight_rest_days = 0.2
weight_intensity = 0.4

# Calculate performance score
df['Performance Score'] = (weight_cumulative_load * df['Cumulative Load'] +
                           weight_rest_days * df['Days Between Activity'] +
                           weight_intensity * df['Training Session Intensity'])


In [122]:
df[df['Performance Score'] == df['Performance Score'].max()]

,Distance,Max Heart Rate,Relative Effort,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Low,Elevation High,Max Grade,Average Grade,Max Cadence,Average Cadence,Average Heart Rate,Calories,Days Between Activity,Training Stress,Training Session Intensity,Cumulative Load,Performance Score
Activity Date,,,,,,,,,,,,,,,,,,,,
2018-05-25 21:19:25,3.47,169.650862,57.633621,1466.0,6.7,2.370396,281.0,-112.599998,217.199997,49.5,0.13813,112.0,73.292191,147.33555,357.70636,220.0,7.041306,0.082346,0.481987,44.225733




---

# **Baseline Model**



---



In [123]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [124]:
# Split the data into features X and target y
# Here we're assuming that 'Training Stress' is the target variable
X = df.drop('Performance Score', axis=1)
y = df['Performance Score']

In [125]:
# Calculate the mean value of the target variable and use it as a constant prediction for every data point.
# This baseline model assumes that all predictions will be equal to the mean value, disregarding any input features.
y.mean()

0.848868716672797

In [126]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [127]:
# Create a baseline model using Linear Regression
lr = LinearRegression()

# Fit the model to the training data
lr.fit(X_train, y_train)

LinearRegression()

In [128]:
# Make predictions on the test data
y_pred = lr.predict(X_test)
y_pred

array([0.64867761, 1.17105594, 0.36253527, 0.2877464 , 0.74118201,
       0.58692738, 0.77696213, 0.3792193 , 0.25420168, 3.48220037,
       1.07451462, 1.24835304, 0.38696061, 0.26967505, 0.45835795,
       0.44633755, 2.95141195, 0.60504727, 1.12613934, 0.41184246,
       0.33547903, 2.52863364, 0.41196345, 0.67459766, 0.51754791,
       0.23417307, 0.49539805, 1.20124007, 0.57509052, 0.45941688,
       0.40151588, 0.36546778, 0.31192453, 0.67795031, 0.32395607,
       0.53040621, 0.30918912, 0.50571663, 0.61425891, 0.64227773,
       0.70186625, 2.31534703, 0.62211705, 0.40762382, 0.36739237,
       0.2545478 , 0.61070501, 0.46419162, 0.37692927, 0.64474398,
       8.94350434, 0.29495624, 0.35299488, 0.82290559, 0.69746479,
       0.22524471, 0.32110406, 1.1641894 , 0.77577349, 1.05521793,
       1.07422889, 0.80502705, 0.40821764, 0.44315456, 0.58502132,
       0.59818071, 0.83324357, 0.90269926, 0.49735743, 0.62642483,
       0.89205226, 1.59542106, 0.22037316, 0.99796054, 1.17391

The coefficient of determination (R-squared) is a statistical measure that indicates the proportion of the variance in the dependent variable (target variable) that can be explained by the independent variables in the regression model. It ranges between 0 and 1, where:

*   R-squared = 1 indicates that the model perfectly predicts the target variable.

*   R-squared = 0 indicates that the model fails to explain any of the variability in the target variable.

An R-squared value closer to 1 suggests that the independent variables in your model explain a larger portion of the variability in the target variable. It indicates a better fit of the model to the data.

However, it's important to note that R-squared alone does not provide information about the correctness of the model or the significance of the independent variables. It doesn't consider the possibility of overfitting or the presence of omitted variables. Therefore, it's crucial to interpret R-squared in conjunction with other evaluation metrics, domain knowledge, and the specific context of your problem.

In [129]:
r_squared = lr.score(X, y)
r_squared

1.0

Mean Squared Error (MSE), it is a common evaluation metric for regression problems, including linear regression. It measures the average of the squares of the errors—that is, the average squared difference between the estimated values and the actual value. The MSE is a measure of the quality of an estimator—it is always non-negative, and values closer to zero are better.

A key property of the MSE is that it penalizes larger errors more than smaller ones, due to the squared term. This makes it a useful metric when large errors are particularly undesirable.

In [130]:
# Evaluate the model
def calculate_mse(actual, predicted):
  mse = mean_squared_error(actual, predicted)
  return mse

mse = calculate_mse(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 8.670591636201176e-30


Root Mean Squared Error (RMSE): This is the square root of the MSE. The RMSE is in the same units as the dependent variable, making it more interpretable than the MSE. It also penalizes large errors more than the MAE.

In [131]:
def calculate_rmse(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    return rmse

# Calculate RMSE
rmse = calculate_rmse(y_test, y_pred)
print(f'RMSE: {rmse}')

RMSE: 2.9445868362473498e-15


On average, the predictions of our linear regression model for 'Perfomance Score' have an error of approximately 0.0000000000000029445868362473498 units (!!). This indicates the typical distance between the predicted 'Performance Score' values and the actual 'Performance Score' values.